In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py


## Parse ITJ process

In [3]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


## Construct PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

In [7]:
debug = 0
verbose = 1
use_partial_order = 1
reset_to_home = 0

In [8]:
pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug, 
                                            reset_to_home=reset_to_home)

In [9]:
print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)

Init: [('RobotConf', Conf-0.93), ('RobotAtConf', Conf-0.93), ('CanMove',), ('RobotToolChangerEmpty',), ('Element', 'b0'), ('AtRack', 'b0'), ('IsElement', 'b0'), ('Grounded', 'b0'), ('Element', 'b1'), ('AtRack', 'b1'), ('IsElement', 'b1'), ('Grounded', 'b1'), ('Element', 'b2'), ('AtRack', 'b2'), ('IsElement', 'b2'), ('Element', 'b3'), ('AtRack', 'b3'), ('IsElement', 'b3'), ('Element', 'b5'), ('AtRack', 'b5'), ('IsElement', 'b5'), ('Grounded', 'b5'), ('Element', 'b6'), ('AtRack', 'b6'), ('IsElement', 'b6'), ('Element', 'b7'), ('AtRack', 'b7'), ('IsElement', 'b7'), ('Grounded', 'b7'), ('Element', 'b8'), ('AtRack', 'b8'), ('IsElement', 'b8'), ('Element', 'b4'), ('AtRack', 'b4'), ('IsElement', 'b4'), ('Joint', 'b0', 'b2'), ('NoToolAtJoint', 'b0', 'b2'), ('Joint', 'b0', 'b3'), ('NoToolAtJoint', 'b0', 'b3'), ('Joint', 'b0', 'b6'), ('NoToolAtJoint', 'b0', 'b6'), ('Joint', 'b1', 'b2'), ('NoToolAtJoint', 'b1', 'b2'), ('Joint', 'b1', 'b3'), ('NoToolAtJoint', 'b1', 'b3'), ('Joint', 'b1', 'b8'), ('

In [47]:
from integral_timber_joints.assembly.beam_assembly_method import BeamAssemblyMethod

beam_seq = process.assembly.sequence
for e in beam_seq:
    if process.assembly.get_assembly_method(e) == BeamAssemblyMethod.GROUND_CONTACT:
        print('Grounded: ', e)
for j in process.assembly.joint_ids():
    print(j)

Grounded:  b0
Grounded:  b1
Grounded:  b5
Grounded:  b7
('b0', 'b2')
('b0', 'b3')
('b0', 'b6')
('b1', 'b2')
('b1', 'b3')
('b1', 'b8')
('b2', 'b0')
('b2', 'b1')
('b3', 'b0')
('b3', 'b1')
('b4', 'b5')
('b4', 'b7')
('b5', 'b4')
('b5', 'b6')
('b6', 'b0')
('b6', 'b5')
('b7', 'b4')
('b7', 'b8')
('b8', 'b1')
('b8', 'b7')


## Solve

In [54]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

pddlstream_problem = get_pddlstream_problem(process, use_partial_order=1, debug=True,
                                            reset_to_home=1)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(pddlstream_problem, algorithm='incremental', 
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'c1'), ('AtRack', 'c2'), ('AtRack', 'c3'), ('AtRack', 'c4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 107 | Solved: False | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Time: 0.000
Iteration: 2 | Complexity: 1 | Calls: 32 | Evaluations: 235 | Solved: False | Cost: inf | Search Time: 0.050 | Sample Time: 0.000 | Time: 0.050
Summary: {complexity: 1, cost: 54.000, evaluations: 235, iterations: 2, length: 54, run_time: 2.099, sample_time: 0.000, search_time: 2.099, solutions: 1, solved: True, timeout: False}
Planning succeeds
----------
 1) pick_gripper_from_rack g1 @conf1813 @conf2813 @traj813
 2) pick_element_from_rack b0 @conf1787 @conf2787 @traj787 g1
 3) place_element_on_structure b0 @conf1788 @co

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)

In [62]:
actions = [action for action in plan if action.name == 'place_element_on_structure']
element_sequence = [ac.args[0] for ac in actions]

In [63]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [64]:
element_sequence

['b0', 'b1', 'b5', 'b7', 'b8', 'b3', 'b2', 'b6', 'b4']

In [ ]:
# TODO visualize sequence in a pybullet window